In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.linalg as la
import scipy.sparse as sparse
from matplotlib import cm
import timeit

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
def u_analytical(x, y):

    uan = np.sin(np.pi*x)*np.sin(np.pi*y)

    return uan

def source(x, y):

    f = -2*np.pi**2*np.sin(np.pi*x)*np.sin(np.pi*y)

    return f

In [4]:
def jacobi_iteration(matrix, source, grid, init_guess=None, tolerance=1.e-5, itermax=50):

    M, f, u0, tol, kmax = matrix, source, init_guess, tolerance, itermax

    x, y = grid

    Nx = x.shape[1] - 1
    Ny = y.shape[0] - 1
    
    dx = (x[0, -1] - x[0, 0])/Nx
    dy = (y[-1, 0] - y[0, 0])/Ny
    

    if init_guess is None:

        u0 = np.zeros_like(f)

    u = u0
    k = 0
    rel_diff = tol +1
    conv_hist = []
    while k < kmax and rel_diff > tol:

        u_next = np.zeros_like(u)

        u_next[1:-1, 1:-1] = ((u[:-2, 1:-1] + u[2:, 1:-1])*dx**2 + (u[1:-1, :-2] + u[1:-1, 2:])*dy**2 -
                              dx**2*dy**2*f[1:-1, 1:-1])/(2*(dx**2+dy**2))

        
        rel_diff = la.norm(u_next - u)/la.norm(u)
        conv_hist.append(rel_diff)

        k += 1

        u = u_next

    return u, k, rel_diff, conv_hist

In [5]:
Nx = 40
Ny = 40

x_I, x_F = 0, 1
y_I, y_F = 0, 1

X = np.linspace(x_I, x_F, Nx+1)
Y = np.linspace(y_I, y_F, Ny+1)

x, y = np.meshgrid(X, Y)

# print(x, y)

In [6]:
f = source(x, y)
u_guess = np.cos(np.pi*x)*np.cos(np.pi*y)

start = timeit.default_timer()

u, k, rel_diff, conv_hist = jacobi_iteration(np.eye(Nx), f, (x, y), init_guess=u_guess, tolerance=1.e-8, itermax=20000)

elapsed = timeit.default_timer() - start

print('Number of iterations: {:} \nRelative difference: {:1.2E} \nElapsed time: {:1.2E}'.format(k, rel_diff, elapsed))



Number of iterations: 4095 
Relative difference: 9.99E-09 
Elapsed time: 4.13E-01


In [7]:
u_an = u_analytical(x, y)

error = la.norm(u - u_an, 2)/la.norm(u_an, 2)

print(error)

0.0005109687754905329


In [8]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')


scat = ax.scatter(x, y, u, c = 'r', label = 'numerical', alpha = 1)
surf = ax.plot_surface(x, y, u_an, cmap = cm.coolwarm, label = 'analytical', alpha = 0.8)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('$u = \sin\pi x \sin\pi y$')

surf._facecolors2d=surf._facecolors3d
surf._edgecolors2d=surf._edgecolors3d
ax.legend(loc = 2)
plt.show()